# **dim_date Table**

Start by creating the table and addin the comment

In [0]:
%sql
CREATE TABLE IF NOT EXISTS bikes.gold.dim_date (
  date DATE COMMENT 'Actual calendar date',
  day INT COMMENT 'Day number in the month (1–31)',
  day_name STRING COMMENT 'Full name of the day (e.g., Monday)',
  month INT COMMENT 'Month number (1–12)',
  month_name STRING COMMENT 'Full name of the month (e.g., January)',
  quarter INT COMMENT 'Quarter of the year (1–4)',
  year INT COMMENT 'Calendar year',
  week_of_year INT COMMENT 'Week number of the year (ISO standard)',
  day_of_year INT COMMENT 'Day number within the year (1–365/366)',
  is_weekend BOOLEAN COMMENT 'Indicates whether the date is a weekend (Saturday or Sunday)'
)
COMMENT 'Date dimension table used for time-based analytics and reporting in the Gold layer';

Generate the data and insert it

In [0]:
%sql
INSERT OVERWRITE bikes.gold.dim_date
SELECT
    d AS date,
    day(d) AS day,
    date_format(d, 'EEEE') AS day_name,
    month(d) AS month,
    date_format(d, 'MMMM') AS month_name,
    quarter(d) AS quarter,
    year(d) AS year,
    weekofyear(d) AS week_of_year,
    dayofyear(d) AS day_of_year,
    CASE 
        WHEN dayofweek(d) IN (1, 7) THEN true 
        ELSE false 
    END AS is_weekend
FROM (
    SELECT explode(
        sequence(
            to_date('2000-01-01'),
            to_date('2030-12-31'),
            interval 1 day
        )
    ) AS d
) t;


Add the tags

In [0]:
%sql
ALTER TABLE bikes.gold.dim_date
SET TAGS (
  'layer' = 'gold',
  'table_type' = 'dimension',
  'domain' = 'date',
  'pii' = 'false'
);
